In [ ]:
import re
import sys

def read_file(file_path):
  read_clauses = []
  with open(file_path, errors='ignore') as input_file:
        for line in input_file:
            line = re.sub(r'\n', '', line)
            line = re.sub(r'[ \t]+$', '', line)
            cl = []
            for c in line.split(" "):
                cl.append(c)
            read_clauses.append(cl)
  return read_clauses

def print_clause(clause_array, clause_no):
    for cl in clause_array:
        print(clause_no, ". ", ' '.join(cl), " { }", sep='')
        clause_no += 1
    return clause_no

def negated_clause(clause):
    for c in range(len(clause)):
        if '~' in clause[c]:
            clause[c] = re.sub(r'~', '', clause[c])
        else:
            clause[c] = '~' + clause[c]
    return clause

def add_to_KB(clause_Number, clause, clause_array):
    clause_array.append([clause])
    print(clause_Number, ". ", ' '.join([clause]), " { }", sep='')
    return clause_array

def main(argv1):
    clauses = []
    clauses = read_file(argv1)
    clause_Number = 1
    
    to_prove = clauses[-1]
    del clauses[-1]

    clause_Number = print_clause(clauses, clause_Number)

    to_prove = negated_clause(to_prove)
    for c in to_prove:
        clauses = add_to_KB(clause_Number, c, clauses)
        clause_Number += 1
    cli = 1
    while cli < clause_Number - 1:
        clj = 0
        while clj < cli:
            #print("testing",cli, clj)
            result = resolve(clauses[cli], clauses[clj], clauses)
            if result is False:
                print(clause_Number, ". ","Contradiction", ' {', cli + 1, ", ", clj + 1, '}', sep='')
                #print(clauses)
                clause_Number += 1
                print("Valid")
                sys.exit(0)
            elif result is True:
                clj += 1
                continue
            else:
                #print("Creating new clause bc result was returned")
                print(clause_Number, ". ",' '.join(result), ' {', cli + 1, ", ", clj + 1, '}', sep='')
                clause_Number += 1
                clauses.append(result)
            clj += 1
        cli += 1
    print('Not Valid')

def resolve(c1, c2, clauses):
    
    resolved2 = c1 + c2
    #print("Resolved 2: " + str(resolved2))
    resolved = None
    hashmap = {}
    for r1 in resolved2:
        if r1 not in hashmap.keys():
            hashmap[r1] = 0

    resolved = list(hashmap.keys())
    disjunction = list(hashmap.keys())
    for literal_1 in c1:
        for literal_2 in c2:
            if neg(literal_1, literal_2):
                resolved.remove(literal_2)
                resolved.remove(literal_1)
                if len(resolved) is 0:
                    return False
                elif imply_True(resolved):
                    return True
                else:
                    for cl in clauses:
                        if symmetric_Difference(resolved, cl) == []:
                            return True
                    return resolved

    if resolved == disjunction: #no resolution, no change
        return True

def imply_True(resolved):
    for r1 in resolved:
        for r2 in resolved:
            if neg(r1, r2):
                return True
    return False

def symmetric_Difference(li1, li2):
    unique_literals = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return unique_literals

def neg(literal_1, literal_2):
    if literal_2 == ('~' + literal_1) or literal_1 == ('~' + literal_2):
        return True
    else:
        return False

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("usage: Python kb_copy.py arg1")
    else:
        main(sys.argv[1])

In [ ]:
main("demo1.in.txt")

1. ~p q { }
2. ~z y { }
3. ~q p { }
4. q ~p z { }
5. q ~p ~z { }
6. z { }
7. ~y { }
8. p { }
9. q ~p y {4, 2}
10. y {6, 2}
11. ~z {7, 2}
12. q {8, 1}
13. q z {8, 4}
14. q ~z {8, 5}
15. q y {9, 8}
16. Contradiction {10, 7}
Valid


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
main("task6.in.txt")

1. ~NoLeak ~LowTemp ReactorUnitSafe ~NoLeakH1 ~NoLeakH2 okH1 okH2 l V1 V2 { }
2. ~NoLeak ~LowTemp ReactorUnitSafe { }
3. ~NoLeakH1 ~NoLeakH2 NoLeak { }
4. ~okH1 NoLeakH1 { }
5. okH1 V1 NoLeakH1 { }
6. ~okH2 NoLeakH2 { }
7. okH2 V2 NoLeakH2 { }
8. ~l ~V1 ~V2 LowTemp { }
9. l ~V1 LowTemp { }
10. l ~V2 LowTemp { }
11. ~l { }
12. ~okH2 { }
13. okH1 { }
14. V1 { }
15. ~V2 { }
16. ~LowTemp { }
17. ~NoLeakH1 ~NoLeakH2 ~LowTemp ReactorUnitSafe okH1 okH2 l V1 V2 {3, 1}
18. ~NoLeakH1 ~NoLeakH2 ~LowTemp ReactorUnitSafe {3, 2}
19. ~okH1 ~NoLeakH2 NoLeak {4, 3}
20. okH1 V1 ~NoLeak ~LowTemp ReactorUnitSafe ~NoLeakH2 okH2 l V2 {5, 1}
21. okH1 V1 ~NoLeakH2 NoLeak {5, 3}
22. V1 NoLeakH1 {5, 4}
23. ~okH2 ~NoLeakH1 NoLeak {6, 3}
24. okH2 V2 ~NoLeak ~LowTemp ReactorUnitSafe ~NoLeakH1 okH1 l V1 {7, 1}
25. okH2 V2 ~NoLeakH1 NoLeak {7, 3}
26. V2 NoLeakH2 {7, 6}
27. ~l ~V1 ~V2 ~NoLeak ReactorUnitSafe {8, 2}
28. ~l ~V2 LowTemp okH1 NoLeakH1 {8, 5}
29. ~l ~V1 LowTemp okH2 NoLeakH2 {8, 7}
30. l ~V1 ~NoLeak React

SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

NameError: ignored